# Data Analytics Must Knows

### Pandas, SQL, PySpark

Situation: 

You are given a JSON data formatted as a string which holds basic employee information in a company that has several departments, and you are given a series of analytics questions to analyze. 

You are to provide your solutions to each question in every language/library - **Pandas**, **SQL**, and **PySpark**.

The data given to you is shown in the next cell:

In [1]:
employees_json = """
    [
        {"EmployeeID":1, "Name":"Alice", "Department":"HR", "Salary":60000, "JoiningDate":"2019-01-15", "PerformanceScore":3},
        {"EmployeeID":2, "Name":"Bob", "Department":"IT", "Salary":70000, "JoiningDate":"2019-06-20", "PerformanceScore":4},
        {"EmployeeID":3, "Name":"Charlie", "Department":"IT", "Salary":80000, "JoiningDate":"2018-07-23", "PerformanceScore":2},
        {"EmployeeID":4, "Name":"David", "Department":"HR", "Salary":65000, "JoiningDate":"2020-02-10", "PerformanceScore":5},
        {"EmployeeID":5, "Name":"Eve", "Department":"Finance", "Salary":75000, "JoiningDate":"2021-03-15", "PerformanceScore":3}
    ]
"""
bonuses_json = """
    [
        {"EmployeeID":1, "Bonus":5000},
        {"EmployeeID":2, "Bonus":7000},
        {"EmployeeID":3, "Bonus":8000},
        {"EmployeeID":6, "Bonus":6000}
    ]
"""

**Q0: Import the necessary libraries and perform the necessary setup to complete the tasks for all the indivated languages and libraries.**

Import libraries

In [2]:
# DateTime and math
import math
from datetime import datetime, date 
# JSON
import json
# Pandas
import pandas as pd
# SQL
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.sql import text
# PySpark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

JSON Parsing

In [3]:
# JSON Parsing (Employee)
employees_dict = json.loads(employees_json)
employees_dict[0]

{'EmployeeID': 1,
 'Name': 'Alice',
 'Department': 'HR',
 'Salary': 60000,
 'JoiningDate': '2019-01-15',
 'PerformanceScore': 3}

In [4]:
# JSON Parsing (Bonuses)
bonuses_dict = json.loads(bonuses_json)
bonuses_dict[0]

{'EmployeeID': 1, 'Bonus': 5000}

Setup Pandas

In [5]:
employees_pd = pd.DataFrame(employees_dict)
employees_pd

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore
0,1,Alice,HR,60000,2019-01-15,3
1,2,Bob,IT,70000,2019-06-20,4
2,3,Charlie,IT,80000,2018-07-23,2
3,4,David,HR,65000,2020-02-10,5
4,5,Eve,Finance,75000,2021-03-15,3


In [6]:
employees_pd.dtypes

EmployeeID           int64
Name                object
Department          object
Salary               int64
JoiningDate         object
PerformanceScore     int64
dtype: object

In [7]:
bonuses_pd = pd.DataFrame(bonuses_dict)
bonuses_pd

,EmployeeID,Bonus
0,1,5000
1,2,7000
2,3,8000
3,6,6000


In [8]:
bonuses_pd.dtypes

EmployeeID    int64
Bonus         int64
dtype: object

Setup PostgreSQL

In [9]:
# Get credentials
load_dotenv()
user = os.environ.get("USER")
pw = os.environ.get("PASS")
db = os.environ.get("DB")
host = os.environ.get("HOST")
api = os.environ.get("API")
port = 5432
schema = 'da_must_knows'

In [10]:
# Connect to database
uri = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}"
alchemyEngine = create_engine(uri)
conn = alchemyEngine.connect()

In [11]:
# Load to SQL
conn.execute(text(f'DROP SCHEMA IF EXISTS {schema} CASCADE'))
conn.execute(text(f'CREATE SCHEMA {schema}'))
employees_pd.to_sql(con=conn,name="employees",schema=schema)
bonuses_pd.to_sql(con=conn,name="bonuses",schema=schema)

4

In [12]:
rs = conn.execute(text(f"SELECT table_name FROM information_schema.tables WHERE table_schema='{schema}'"))
tables = [table[0] for table in rs.fetchall()]
print(f"The tables in the database are: {', '.join(tables)}")

The tables in the database are: employees, bonuses


In [13]:
for table in tables:
    print("=================================")
    print(f'Table [{table}]')
    df = pd.read_sql_query(f'SELECT * FROM {schema}.{table} LIMIT 5', conn)
    print(f'Dimensions: {df.shape[0]} rows x {df.shape[1]} columns\n')
    print(df.head())
    info_df = pd.DataFrame.from_dict({'Datatypes':df.dtypes, 'NULL count':df.isna().sum()})
    print()
    print(info_df)
    print()

Table [employees]
Dimensions: 5 rows x 7 columns

   index  EmployeeID     Name Department  Salary JoiningDate  PerformanceScore
0      0           1    Alice         HR   60000  2019-01-15                 3
1      1           2      Bob         IT   70000  2019-06-20                 4
2      2           3  Charlie         IT   80000  2018-07-23                 2
3      3           4    David         HR   65000  2020-02-10                 5
4      4           5      Eve    Finance   75000  2021-03-15                 3

                 Datatypes  NULL count
index                int64           0
EmployeeID           int64           0
Name                object           0
Department          object           0
Salary               int64           0
JoiningDate         object           0
PerformanceScore     int64           0

Table [bonuses]
Dimensions: 4 rows x 3 columns

   index  EmployeeID  Bonus
0      0           1   5000
1      1           2   7000
2      2           3   8000
3 

Setup PySpark

In [14]:
spark = SparkSession.builder.getOrCreate()

In [15]:
employees_ps = spark.createDataFrame(employees_pd)
employees_ps.show()
employees_ps.printSchema()

+----------+-------+----------+------+-----------+----------------+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|
+----------+-------+----------+------+-----------+----------------+
|         1|  Alice|        HR| 60000| 2019-01-15|               3|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|
|         4|  David|        HR| 65000| 2020-02-10|               5|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|
+----------+-------+----------+------+-----------+----------------+

root
 |-- EmployeeID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- JoiningDate: string (nullable = true)
 |-- PerformanceScore: long (nullable = true)



In [16]:
bonuses_ps = spark.createDataFrame(bonuses_pd)
bonuses_ps.show()
bonuses_ps.printSchema()

+----------+-----+
|EmployeeID|Bonus|
+----------+-----+
|         1| 5000|
|         2| 7000|
|         3| 8000|
|         6| 6000|
+----------+-----+

root
 |-- EmployeeID: long (nullable = true)
 |-- Bonus: long (nullable = true)



**Q1: How can you calculate the average salary for each department?**

In [17]:
# Pandas solution
average_salary = employees_pd.groupby('Department')['Salary'].mean().reset_index()
average_salary

,Department,Salary
0,Finance,75000.0
1,HR,62500.0
2,IT,75000.0


In [18]:
# SQL Solution
average_salary = pd.read_sql_query(f'''
    SELECT "Department", AVG("Salary") AS "Salary"
    FROM {schema}.employees
    GROUP BY "Department"
''',conn)
average_salary

,Department,Salary
0,Finance,75000.0
1,IT,75000.0
2,HR,62500.0


In [19]:
# PySpark Solution
employees_ps.groupBy('Department').avg("salary").alias("avg_salary").show(truncate=False)

+----------+-----------+
|Department|avg(salary)|
+----------+-----------+
|HR        |62500.0    |
|IT        |75000.0    |
|Finance   |75000.0    |
+----------+-----------+



**Q2: Determine the employee with the highest performance score in each department.**

In [20]:
# Pandas solution
x = employees_pd.groupby('Department')['PerformanceScore'].idxmax()
highest_performers = employees_pd.loc[x]
highest_performers

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore
4,5,Eve,Finance,75000,2021-03-15,3
3,4,David,HR,65000,2020-02-10,5
1,2,Bob,IT,70000,2019-06-20,4


In [21]:
# SQL Solution
highest_performers = pd.read_sql_query(f'''
    WITH rank_per_department AS (
        SELECT *, RANK() OVER 
            (PARTITION BY "Department" ORDER BY "PerformanceScore" DESC) AS "rank"
        FROM {schema}.employees
    )
    SELECT *
    FROM rank_per_department
    WHERE rank = 1       
''',conn)
highest_performers

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,rank
0,4,5,Eve,Finance,75000,2021-03-15,3,1
1,3,4,David,HR,65000,2020-02-10,5,1
2,1,2,Bob,IT,70000,2019-06-20,4,1


In [22]:
# PySpark Solution
windowSpec = Window.partitionBy("department").orderBy("PerformanceScore")
ranks = employees_ps.withColumn("rank",F.rank().over(windowSpec))
highest_performers = ranks.filter('rank == 1')
highest_performers.show()

+----------+-------+----------+------+-----------+----------------+----+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|rank|
+----------+-------+----------+------+-----------+----------------+----+
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|   1|
|         1|  Alice|        HR| 60000| 2019-01-15|               3|   1|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|   1|
+----------+-------+----------+------+-----------+----------------+----+



**Q3: Add a new column that represents the number of years each employee has been with the company based on the JoiningDate.**

In [23]:
# Pandas solution
employees_pd['JoiningDate'] = pd.to_datetime(employees_pd['JoiningDate'])
current_date = datetime.now()
employees_pd['YearsWithCompany'] = (current_date - employees_pd['JoiningDate']).dt.days // 365
employees_pd

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany
0,1,Alice,HR,60000,2019-01-15,3,5
1,2,Bob,IT,70000,2019-06-20,4,5
2,3,Charlie,IT,80000,2018-07-23,2,6
3,4,David,HR,65000,2020-02-10,5,4
4,5,Eve,Finance,75000,2021-03-15,3,3


In [24]:
# SQL solution
conn.execute(text(f'''
    ALTER TABLE {schema}.employees
    ADD COLUMN "YearsWithCompany" INT;
'''))
conn.execute(text(f'''
    UPDATE {schema}.employees
    SET "YearsWithCompany" = DATE_PART('year', CURRENT_DATE) - DATE_PART('year', CAST("JoiningDate" AS DATE))
'''))
pd.read_sql_query(f'SELECT * FROM {schema}.employees',conn)

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany
0,0,1,Alice,HR,60000,2019-01-15,3,5
1,1,2,Bob,IT,70000,2019-06-20,4,5
2,2,3,Charlie,IT,80000,2018-07-23,2,6
3,3,4,David,HR,65000,2020-02-10,5,4
4,4,5,Eve,Finance,75000,2021-03-15,3,3


In [25]:
# PySpark solution
employees_ps = employees_ps.withColumn("YearsWithCompany", F.floor(F.datediff(F.current_date(), F.to_date(F.col("JoiningDate"))) / 365))
employees_ps.show()

+----------+-------+----------+------+-----------+----------------+----------------+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|YearsWithCompany|
+----------+-------+----------+------+-----------+----------------+----------------+
|         1|  Alice|        HR| 60000| 2019-01-15|               3|               5|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|               5|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|               6|
|         4|  David|        HR| 65000| 2020-02-10|               5|               4|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|               3|
+----------+-------+----------+------+-----------+----------------+----------------+



**Q4: Create a pivot table to display the total salary and average performance score for each department.**

In [26]:
# Pandas solution
pivot_table = pd.pivot_table(employees_pd, values=['Salary','PerformanceScore'], index='Department', aggfunc={'Salary':'sum','PerformanceScore':'mean'})
pivot_table

,PerformanceScore,Salary
Department,,
Finance,3.0,75000
HR,4.0,125000
IT,3.0,150000


In [27]:
# SQL Solution
pd.read_sql_query(f'''
    SELECT "Department", AVG("PerformanceScore") AS "PerformanceScore", SUM("Salary") AS "Salary"
    FROM {schema}.employees
    GROUP BY "Department"
''',conn)

,Department,PerformanceScore,Salary
0,Finance,3.0,75000.0
1,IT,3.0,150000.0
2,HR,4.0,125000.0


In [28]:
# PySpark solution
employees_ps.groupBy('Department').agg(F.avg("PerformanceScore").alias("PerformanceScore"),F.sum("Salary").alias("Salary")).show()

+----------+----------------+------+
|Department|PerformanceScore|Salary|
+----------+----------------+------+
|        HR|             4.0|125000|
|        IT|             3.0|150000|
|   Finance|             3.0| 75000|
+----------+----------------+------+



**Q5: Create a new DataFrame containing only the employees from the IT department who have a performance score greater than 3.**

In [29]:
# Pandas solution
IT_high_performance = employees_pd[(employees_pd['Department']=='IT') & (employees_pd['PerformanceScore'] > 3)]
IT_high_performance

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany
1,2,Bob,IT,70000,2019-06-20,4,5


In [30]:
# SQL solution
conn.execute(text(f'''
    DROP TABLE IF EXISTS {schema}.it_high_performance;
    CREATE TABLE {schema}.it_high_performance AS
    SELECT *
    FROM {schema}.employees
    WHERE "Department"='IT' AND "PerformanceScore" > 3;
'''))
pd.read_sql_query(f"SELECT * FROM {schema}.it_high_performance", conn)

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany
0,1,2,Bob,IT,70000,2019-06-20,4,5


In [31]:
# PySpark solution
IT_high_performance_ps = employees_ps.filter('Department=="IT" AND PerformanceScore > 3')
IT_high_performance_ps.show()

+----------+----+----------+------+-----------+----------------+----------------+
|EmployeeID|Name|Department|Salary|JoiningDate|PerformanceScore|YearsWithCompany|
+----------+----+----------+------+-----------+----------------+----------------+
|         2| Bob|        IT| 70000| 2019-06-20|               4|               5|
+----------+----+----------+------+-----------+----------------+----------------+



**Q6: Perform an merge of the employees data with the bonuses data based on EmployeeID and keep only the employees that exist in the employees data. Impute the missing values appropriately.**

In [32]:
# Pandas solution
merged_pd = employees_pd.merge(bonuses_pd,on='EmployeeID',how='left')
merged_pd = merged_pd.fillna(0)
merged_pd

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,Bonus
0,1,Alice,HR,60000,2019-01-15,3,5,5000.0
1,2,Bob,IT,70000,2019-06-20,4,5,7000.0
2,3,Charlie,IT,80000,2018-07-23,2,6,8000.0
3,4,David,HR,65000,2020-02-10,5,4,0.0
4,5,Eve,Finance,75000,2021-03-15,3,3,0.0


In [33]:
# SQL solution
pd.read_sql_query(f'''
    SELECT "EmployeeID","Name","Department","Salary","JoiningDate","PerformanceScore","YearsWithCompany",COALESCE("Bonus",0) AS "Bonus"
    FROM {schema}.employees LEFT JOIN {schema}.bonuses USING("EmployeeID")
''',conn)

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,Bonus
0,1,Alice,HR,60000,2019-01-15,3,5,5000
1,2,Bob,IT,70000,2019-06-20,4,5,7000
2,3,Charlie,IT,80000,2018-07-23,2,6,8000
3,5,Eve,Finance,75000,2021-03-15,3,3,0
4,4,David,HR,65000,2020-02-10,5,4,0


In [34]:
# PySpark solution
merged_ps = employees_ps.join(bonuses_ps, employees_ps.EmployeeID == bonuses_ps.EmployeeID, 'left').drop(bonuses_ps.EmployeeID)
merged_ps = merged_ps.na.fill({'Bonus' : 0})
merged_ps.show()

+----------+-------+----------+------+-----------+----------------+----------------+-----+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|YearsWithCompany|Bonus|
+----------+-------+----------+------+-----------+----------------+----------------+-----+
|         1|  Alice|        HR| 60000| 2019-01-15|               3|               5| 5000|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|               5| 7000|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|               6| 8000|
|         4|  David|        HR| 65000| 2020-02-10|               5|               4|    0|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|               3|    0|
+----------+-------+----------+------+-----------+----------------+----------------+-----+



**Q7: Calculate the cumulative sum of the Salary column grouped by Department, ordered by joining date.**

In [35]:
# Pandas solution
employees_pd = employees_pd.sort_values('JoiningDate')
employees_pd['CumSalary'] = employees_pd.groupby('Department')['Salary'].cumsum()
employees_pd


,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,CumSalary
2,3,Charlie,IT,80000,2018-07-23,2,6,80000
0,1,Alice,HR,60000,2019-01-15,3,5,60000
1,2,Bob,IT,70000,2019-06-20,4,5,150000
3,4,David,HR,65000,2020-02-10,5,4,125000
4,5,Eve,Finance,75000,2021-03-15,3,3,75000


In [36]:
# SQL solution
pd.read_sql_query(f'''
    SELECT *, SUM("Salary") OVER(PARTITION BY "Department" ORDER BY "JoiningDate" ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS "CumSalary"
    FROM {schema}.employees
    ORDER BY "JoiningDate"
''',conn)

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,CumSalary
0,2,3,Charlie,IT,80000,2018-07-23,2,6,80000.0
1,0,1,Alice,HR,60000,2019-01-15,3,5,60000.0
2,1,2,Bob,IT,70000,2019-06-20,4,5,150000.0
3,3,4,David,HR,65000,2020-02-10,5,4,125000.0
4,4,5,Eve,Finance,75000,2021-03-15,3,3,75000.0


In [37]:
# PySpark solution
windowspec2 = (Window.partitionBy('Department').orderBy('JoiningDate')
             .rangeBetween(Window.unboundedPreceding, 0))
employees_ps = employees_ps.withColumn('CumSalary',F.sum('Salary').over(windowspec2))
employees_ps.orderBy('JoiningDate').show()

+----------+-------+----------+------+-----------+----------------+----------------+---------+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|YearsWithCompany|CumSalary|
+----------+-------+----------+------+-----------+----------------+----------------+---------+
|         3|Charlie|        IT| 80000| 2018-07-23|               2|               6|    80000|
|         1|  Alice|        HR| 60000| 2019-01-15|               3|               5|    60000|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|               5|   150000|
|         4|  David|        HR| 65000| 2020-02-10|               5|               4|   125000|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|               3|    75000|
+----------+-------+----------+------+-----------+----------------+----------------+---------+



**Q8: Rank the employees within each department based on their Salary.**

In [38]:
# Pandas solution
employees_pd['SalaryRank'] = employees_pd.groupby('Department')['Salary'].rank(ascending=False)
employees_pd = employees_pd.sort_index()
employees_pd

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,CumSalary,SalaryRank
0,1,Alice,HR,60000,2019-01-15,3,5,60000,2.0
1,2,Bob,IT,70000,2019-06-20,4,5,150000,2.0
2,3,Charlie,IT,80000,2018-07-23,2,6,80000,1.0
3,4,David,HR,65000,2020-02-10,5,4,125000,1.0
4,5,Eve,Finance,75000,2021-03-15,3,3,75000,1.0


In [39]:
# SQL solution
pd.read_sql_query(f'''
    SELECT *, RANK() OVER(PARTITION BY "Department" ORDER BY "Salary" DESC) AS "SalaryRank"
    FROM {schema}.employees
    ORDER BY index
''',conn)

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,YearsWithCompany,SalaryRank
0,0,1,Alice,HR,60000,2019-01-15,3,5,2
1,1,2,Bob,IT,70000,2019-06-20,4,5,2
2,2,3,Charlie,IT,80000,2018-07-23,2,6,1
3,3,4,David,HR,65000,2020-02-10,5,4,1
4,4,5,Eve,Finance,75000,2021-03-15,3,3,1


In [40]:
# PySpark solution
windowspec3 = Window.partitionBy('Department').orderBy(F.desc('Salary'))
employees_ps = employees_ps.withColumn('SalaryRank', F.rank().over(windowspec3))
employees_ps.orderBy('EmployeeID').show()

+----------+-------+----------+------+-----------+----------------+----------------+---------+----------+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|YearsWithCompany|CumSalary|SalaryRank|
+----------+-------+----------+------+-----------+----------------+----------------+---------+----------+
|         1|  Alice|        HR| 60000| 2019-01-15|               3|               5|    60000|         2|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|               5|   150000|         2|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|               6|    80000|         1|
|         4|  David|        HR| 65000| 2020-02-10|               5|               4|   125000|         1|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|               3|    75000|         1|
+----------+-------+----------+------+-----------+----------------+----------------+---------+----------+



**Q9: Show the count of employees per department who have been with the company for more than 4 years. Include departments with no such employees. Order the result by descending order of count of employees meeting the criteria.**

In [41]:
# Pandas solution
tenure_counts = employees_pd.groupby('Department')['YearsWithCompany'].apply(lambda num: (num > 4).sum()).reset_index(name='Count').sort_values('Count',ascending=False)
tenure_counts

,Department,Count
2,IT,2
1,HR,1
0,Finance,0


In [42]:
# SQL solution
pd.read_sql_query(f'''
    SELECT "Department", COUNT(CASE WHEN "YearsWithCompany" > 4 THEN TRUE ELSE NULL END) AS "Count"
    FROM {schema}.employees
    GROUP BY "Department"
    ORDER BY "Count" DESC
''',conn)

,Department,Count
0,IT,2
1,HR,1
2,Finance,0


In [43]:
# PySpark solution
count_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
tenure_counts = employees_ps.groupBy('Department').agg(count_cond(F.col('YearsWithCompany') > 4).alias('Count'))
tenure_counts.orderBy(F.desc('Count')).show()

+----------+-----+
|Department|Count|
+----------+-----+
|        IT|    2|
|        HR|    1|
|   Finance|    0|
+----------+-----+



**Q10: (A) Calculate the total salary dispensed by the company each year for the years 2018 until 2024.**

**Assume that salaries are paid at the end of the year equivalent to the amount indicated in the 'Salary' column if the employee is able to work for that full year, or a fractional amount of the salary (floored) depending on the length worked by the employee on his starting year (e.g. Joining date: 2020-12-01 -> floor(Salary * 30 / 365) -> since there are 30 days left till the end of the year).**

**(B) Create a pivot table of the total annual salaries dispensed in (A) by breaking down the totals by department.**

Part A

In [44]:
# Pandas solution
years = [year for year in range(2018,2025)]
salaries_dict = {}
for year in years:
    salaries_dict[year] = 0.0

for i, row in employees_pd.iterrows():
    key = row['JoiningDate'].year
    salaries_dict[key] += math.floor((date(key,12,31).timetuple().tm_yday - row['JoiningDate'].timetuple().tm_yday) / 365 * row['Salary'])
    while key < 2024:
        key += 1
        salaries_dict[key] += row['Salary']
pd.DataFrame(salaries_dict, index=['Total Salary']).T

,Total Salary
2018,35287.0
2019,174739.0
2020,267876.0
2021,334794.0
2022,350000.0
2023,350000.0
2024,350000.0


In [45]:
# SQL Solution Part 1 (Creating a table for EmployeeID and Year combinations)
pd.read_sql_query(f'''
        SELECT 
            DISTINCT "EmployeeID",
            '2018-01-01'::DATE + GENERATE_SERIES(0,6) * INTERVAL '1 YEAR' AS "StartYear"
        FROM
            {schema}.employees 
        ORDER BY "EmployeeID", "StartYear"
        LIMIT 15
''',conn)

,EmployeeID,StartYear
0,1,2018-01-01
1,1,2019-01-01
2,1,2020-01-01
3,1,2021-01-01
4,1,2022-01-01
5,1,2023-01-01
6,1,2024-01-01
7,2,2018-01-01
8,2,2019-01-01
9,2,2020-01-01


In [46]:
# SQL Solution Part 2 (Joining Part 1 table and Employees table and computing for annual salary based on conditions)
pd.read_sql_query(f'''
    WITH empid_year_cte AS (
        SELECT 
            DISTINCT "EmployeeID",
            '2018-01-01'::DATE + GENERATE_SERIES(0,6) * INTERVAL '1 YEAR' AS "StartYear"
        FROM
            {schema}.employees 
    )
    SELECT 
        "EmployeeID", "StartYear", "JoiningDate", "Salary",
        CASE WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") >= 1 THEN 0
                WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") = 0 THEN 
                    FLOOR(DATE_PART('day', "StartYear" + INTERVAL '1 YEAR' - INTERVAL '1 DAY' - "JoiningDate"::DATE) / 365.0 * "Salary")
                ELSE "Salary" END AS "Earned"
    FROM empid_year_cte 
    JOIN {schema}.employees USING ("EmployeeID")
    ORDER BY "EmployeeID", "StartYear"
    LIMIT 15
''',conn)

,EmployeeID,StartYear,JoiningDate,Salary,Earned
0,1,2018-01-01,2019-01-15,60000,0.0
1,1,2019-01-01,2019-01-15,60000,57534.0
2,1,2020-01-01,2019-01-15,60000,60000.0
3,1,2021-01-01,2019-01-15,60000,60000.0
4,1,2022-01-01,2019-01-15,60000,60000.0
5,1,2023-01-01,2019-01-15,60000,60000.0
6,1,2024-01-01,2019-01-15,60000,60000.0
7,2,2018-01-01,2019-06-20,70000,0.0
8,2,2019-01-01,2019-06-20,70000,37205.0
9,2,2020-01-01,2019-06-20,70000,70000.0


In [47]:
# SQL Solution Part 3 (Computing total salary by year)
pd.read_sql_query(f'''
    WITH empid_year_cte AS (
        SELECT 
            DISTINCT "EmployeeID",
            '2018-01-01'::DATE + GENERATE_SERIES(0,6) * INTERVAL '1 YEAR' AS "StartYear"
        FROM
            {schema}.employees 
    ),
    earned_per_employee_year_cte AS (
        SELECT 
            "EmployeeID", "StartYear", "JoiningDate", "Salary",
            CASE WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") >= 1 THEN 0
                 WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") = 0 THEN 
                      FLOOR(DATE_PART('day', "StartYear" + INTERVAL '1 YEAR' - INTERVAL '1 DAY' - "JoiningDate"::DATE) / 365.0 * "Salary")
                 ELSE "Salary" END AS "Earned"
        FROM empid_year_cte 
        JOIN {schema}.employees USING ("EmployeeID")
    )
    SELECT 
        DATE_PART('year', "StartYear")::INT AS "Year",
        SUM("Earned") AS "Total Salary"
    FROM
        earned_per_employee_year_cte
    GROUP BY "Year"
    ORDER BY "Year"
''',conn)

,Year,Total Salary
0,2018,35287.0
1,2019,174739.0
2,2020,267876.0
3,2021,334794.0
4,2022,350000.0
5,2023,350000.0
6,2024,350000.0


In [48]:
# PySpark solution Part 1
# Note: Sums will be in columns, since for each employee info, we return a row where each column represents his or her salary earned for the year
from pyspark.sql.types import *

def getSalariesPerYear(row):
    salaries = []
    join_date = datetime.strptime(row.JoiningDate, '%Y-%m-%d').date()
    for year in range(2018,2025):
        if join_date.year - year >= 1:
            salaries.append(0)
        elif join_date.year - year == 0:
            salaries.append(math.floor((date(year,12,31).timetuple().tm_yday - join_date.timetuple().tm_yday) / 365 * row.Salary))
        else:
            salaries.append(row.Salary)
    return tuple(salaries)
            
salaries_rdd = employees_ps.rdd.map(getSalariesPerYear)
salaries_rdd.take(5)

[(0, 0, 0, 59794, 75000, 75000, 75000),
 (0, 0, 57876, 65000, 65000, 65000, 65000),
 (0, 57534, 60000, 60000, 60000, 60000, 60000),
 (35287, 80000, 80000, 80000, 80000, 80000, 80000),
 (0, 37205, 70000, 70000, 70000, 70000, 70000)]

In [49]:
# PySpark solution Part 2
col_names = [str(year) for year in range(2018,2025)]
salaries_df = salaries_rdd.toDF(col_names)
salaries_df.show()

+-----+-----+-----+-----+-----+-----+-----+
| 2018| 2019| 2020| 2021| 2022| 2023| 2024|
+-----+-----+-----+-----+-----+-----+-----+
|    0|    0|    0|59794|75000|75000|75000|
|    0|    0|57876|65000|65000|65000|65000|
|    0|57534|60000|60000|60000|60000|60000|
|35287|80000|80000|80000|80000|80000|80000|
|    0|37205|70000|70000|70000|70000|70000|
+-----+-----+-----+-----+-----+-----+-----+



In [50]:
# PySpark solution Part 3
total_salaries = salaries_df.select(
    F.sum('2018').alias('2018'),
    F.sum('2019').alias('2019'),
    F.sum('2020').alias('2020'),
    F.sum('2021').alias('2021'),
    F.sum('2022').alias('2022'),
    F.sum('2023').alias('2023'),
    F.sum('2024').alias('2024'))
total_salaries.show()

+-----+------+------+------+------+------+------+
| 2018|  2019|  2020|  2021|  2022|  2023|  2024|
+-----+------+------+------+------+------+------+
|35287|174739|267876|334794|350000|350000|350000|
+-----+------+------+------+------+------+------+



Part B

In [51]:
# Pandas solution 
years = [year for year in range(2018,2025)]
depts = employees_pd['Department'].unique().tolist()

year_dept_salaries = {}
for year in years:
    for dept in depts:
        year_dept_salaries[(year,dept)] = 0

for i, row in employees_pd.iterrows():
    key = row['JoiningDate'].year
    dept = row['Department']
    year_dept_salaries[(key,dept)] += math.floor((date(key,12,31).timetuple().tm_yday - row['JoiningDate'].timetuple().tm_yday) / 365 * row['Salary'])
    while key < 2024:
        key += 1
        year_dept_salaries[(key,dept)] += row['Salary'] 

salaries_list = []
for key, value in year_dept_salaries.items():
    salaries_list.append((key[0], key[1], value))

salaries_df = pd.DataFrame(salaries_list, columns=['Year', 'Department', 'Salary'])
salaries_df.pivot_table(index=['Year'],columns=['Department'],values='Salary')

Department,Finance,HR,IT
Year,,,
2018,0.0,0.0,35287.0
2019,0.0,57534.0,117205.0
2020,0.0,117876.0,150000.0
2021,59794.0,125000.0,150000.0
2022,75000.0,125000.0,150000.0
2023,75000.0,125000.0,150000.0
2024,75000.0,125000.0,150000.0


In [52]:
# SQL Solution
pd.read_sql_query(f'''
    WITH empid_year_cte AS (
        SELECT 
            DISTINCT "EmployeeID",
            '2018-01-01'::DATE + GENERATE_SERIES(0,6) * INTERVAL '1 YEAR' AS "StartYear"
        FROM
            {schema}.employees 
    ),
    earned_per_employee_year_cte AS (
        SELECT 
            "EmployeeID", "StartYear", "JoiningDate", "Department", "Salary",
            CASE WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") >= 1 THEN 0
                 WHEN DATE_PART('year', "JoiningDate"::DATE) - DATE_PART('year', "StartYear") = 0 THEN 
                      FLOOR(DATE_PART('day', "StartYear" + INTERVAL '1 YEAR' - INTERVAL '1 DAY' - "JoiningDate"::DATE) / 365.0 * "Salary")
                 ELSE "Salary" END AS "Earned"
        FROM empid_year_cte 
        JOIN {schema}.employees USING ("EmployeeID")
    )
    SELECT 
        DATE_PART('year', "StartYear")::INT AS "Year",
        SUM(CASE WHEN "Department" = 'Finance' THEN "Earned" END) AS "Finance",
        SUM(CASE WHEN "Department" = 'HR' THEN "Earned" END) AS "HR",
        SUM(CASE WHEN "Department" = 'IT' THEN "Earned" END) AS "IT"
    FROM
        earned_per_employee_year_cte
    GROUP BY "Year"
    ORDER BY "Year"
''',conn)

,Year,Finance,HR,IT
0,2018,0.0,0.0,35287.0
1,2019,0.0,57534.0,117205.0
2,2020,0.0,117876.0,150000.0
3,2021,59794.0,125000.0,150000.0
4,2022,75000.0,125000.0,150000.0
5,2023,75000.0,125000.0,150000.0
6,2024,75000.0,125000.0,150000.0


In [53]:
# PySPark Solution
years = [year for year in range(2018,2025)]
depts = employees_pd['Department'].unique().tolist()

year_dept_salaries = {}
for year in years:
    for dept in depts:
        year_dept_salaries[(year,dept)] = 0

for row in employees_ps.rdd.collect():
    join_date = join_date = datetime.strptime(row.JoiningDate, '%Y-%m-%d').date()
    key = join_date.year
    dept = row['Department']
    year_dept_salaries[(key,dept)] += math.floor((date(key,12,31).timetuple().tm_yday - join_date.timetuple().tm_yday) / 365 * row['Salary'])
    while key < 2024:
        key += 1
        year_dept_salaries[(key,dept)] += row['Salary'] 

salaries_list = []
for key, value in year_dept_salaries.items():
    salaries_list.append((key[0], key[1], value))

salaries_df = spark.createDataFrame(salaries_list, ['Year', 'Department', 'Salary'])
salaries_pivot = salaries_df.groupBy("Year").pivot("Department", ["Finance", "HR", "IT"]).sum("Salary")
salaries_pivot.orderBy('Year').show()

+----+-------+------+------+
|Year|Finance|    HR|    IT|
+----+-------+------+------+
|2018|      0|     0| 35287|
|2019|      0| 57534|117205|
|2020|      0|117876|150000|
|2021|  59794|125000|150000|
|2022|  75000|125000|150000|
|2023|  75000|125000|150000|
|2024|  75000|125000|150000|
+----+-------+------+------+

